<a href="https://colab.research.google.com/github/Zatania/Thesis-Semantics-Search-using-SBERT-with-FAISS/blob/main/Brute_Force_Cosine_Distance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install faiss-gpu
!pip install -U sentence-transformers
!pip install torch
!pip install pandas
!pip install numpy
!pip install matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85.5 MB 117 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.0 MB/s 
     |████████████████████████████████| 5.5 MB 23.5 MB/s 
     |████████████████████████████████| 1.3 MB 52.9 MB/s 
     |████████████████████████████████| 182 kB 71.7 MB/s 
     |████████████████████████████████| 7.6 MB 63.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=8cae6d59c1c3d68ea543feb3b51b26c89bf9af20522598a19950d2e6a40fbf6e
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

%cd gdrive/MyDrive/Colab\ Notebooks/Data/CSV

Mounted at /content/gdrive/
/content/gdrive/MyDrive/Colab Notebooks/Data/CSV


In [ ]:
from timeit import default_timer as timer
import numpy as np
import pandas as pd
import faiss
import torch
from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cdist as scipy_cdist
import matplotlib.pyplot as plt

np.random.seed(0)

In [ ]:
contents = pd.read_csv('documents.csv', usecols=['NAME_DOCUMENT', 'CONTENT'])
contents.reset_index(drop=True, inplace=True)

In [ ]:
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = SentenceTransformer('msmarco-distilbert-base-dot-prod-v3', device=torch_device)

documents_embeddings = model.encode(contents.CONTENT.tolist(), device=torch_device, show_progress_bar=True) 

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/115 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.35k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/376 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/341 [00:00<?, ?B/s]

Batches:   0%|          | 0/9 [00:00<?, ?it/s]

In [ ]:
def search(query):
  query_embed = model.encode([query], device=torch_device)

  start = timer()

  similarities = 1 - scipy_cdist(query_embed, documents_embeddings, 'cosine')

  similarities = np.around(similarities, decimals=2)

  end = timer()

  print(f'Finished in "{(end-start):.4f} seconds."')
  sim_list = similarities.tolist()

  print(sim_list)

  val_arr = []
  val_idx = []
  for index, value in enumerate(sim_list[0]):
    if (value > 0.4):
      val_idx.append(index)
      val_arr.append(value)
    else:
      continue

  for score, idx in zip(val_arr,val_idx):
    sim_docs = contents.NAME_DOCUMENT.loc[idx]

    print(f'"Name of Document: "{sim_docs}" || Score: "{score}"')

  print(val_idx)
  print(val_arr)

In [ ]:
query = 'Networking and routing'

search(query)

Finished in "0.0008 seconds."
[[0.15, 0.16, 0.17, 0.22, 0.13, 0.14, 0.16, 0.3, 0.26, 0.2, 0.07, 0.15, 0.22, 0.21, 0.22, 0.21, 0.15, 0.3, 0.21, 0.21, 0.17, 0.15, 0.15, 0.25, 0.18, 0.22, 0.17, 0.11, 0.17, 0.09, 0.17, 0.24, 0.16, 0.14, 0.17, 0.11, 0.1, 0.1, 0.18, 0.18, 0.27, 0.3, 0.26, 0.21, 0.24, 0.21, 0.22, 0.21, 0.14, 0.29, 0.24, 0.2, 0.24, 0.29, 0.23, 0.26, 0.13, 0.13, 0.19, 0.19, 0.32, 0.28, 0.25, 0.28, 0.31, 0.23, 0.2, 0.24, 0.2, 0.3, 0.19, 0.2, 0.19, 0.32, 0.26, 0.18, 0.21, 0.13, 0.23, 0.2, 0.21, 0.27, 0.19, 0.18, 0.14, 0.17, 0.26, 0.29, 0.28, 0.17, 0.24, 0.21, 0.36, 0.19, 0.27, 0.14, 0.23, 0.12, 0.26, 0.27, 0.16, 0.23, 0.32, 0.21, 0.28, 0.22, 0.29, 0.27, 0.1, 0.19, 0.25, 0.2, 0.33, 0.2, 0.2, 0.09, 0.1, 0.16, 0.15, 0.25, 0.16, 0.2, 0.05, 0.11, 0.25, 0.13, 0.18, 0.19, 0.2, 0.23, 0.15, 0.22, 0.2, 0.25, 0.18, 0.19, 0.15, 0.2, 0.13, 0.14, 0.2, 0.21, 0.19, 0.16, 0.25, 0.16, 0.19, 0.12, 0.27, 0.21, 0.15, 0.12, 0.07, 0.24, 0.19, 0.18, 0.22, 0.21, 0.21, 0.2, 0.17, 0.19, 0.25, 0.18, 0.25, 0